In [1]:
import requests
import pandas as pd

def get_crypto_price(symbol, exchange, interval, start_date ):
    api_key = 'YOUR API KEY'
    api_url = f'https://www.alphavantage.co/query?function=CRYPTO_INTRADAY&symbol={symbol}&market={exchange}&interval={interval}&outputsize=full&apikey={api_key}'
    raw_df = requests.get(api_url).json()
#     display(raw_df)
    df = pd.DataFrame(raw_df['Time Series Crypto (60min)']).T
    df = df.rename(columns = {'1. open': 'open', '2. high': 'high', '3. low': 'low', '4. close': 'close', '5. volume': 'volume'})
    for i in df.columns:
        df[i] = df[i].astype(float)
    if start_date:
        df = df[df.index >= start_date]
    return df

btc = get_crypto_price(symbol = 'XRP', exchange = 'USD', interval = "60min", start_date = '2022-07-20')
btc= btc.rename_axis('date').reset_index()
display(btc)
# Save dataframe as CSV
btc.to_csv('crypto.csv', index=False)

,date,open,high,low,close,volume
0,2022-09-24 23:00:00,0.4869,0.4909,0.4864,0.4894,4089069.0
1,2022-09-24 22:00:00,0.4945,0.4946,0.4824,0.4870,29456214.0
2,2022-09-24 21:00:00,0.5036,0.5085,0.4817,0.4946,45911444.0
3,2022-09-24 20:00:00,0.4979,0.5062,0.4959,0.5036,22271544.0
4,2022-09-24 19:00:00,0.5022,0.5029,0.4933,0.4980,26514808.0
...,...,...,...,...,...,...
995,2022-08-14 12:00:00,0.3850,0.3862,0.3834,0.3844,6493096.0
996,2022-08-14 11:00:00,0.3871,0.3886,0.3849,0.3850,8982403.0
997,2022-08-14 10:00:00,0.3906,0.3908,0.3870,0.3870,10021080.0
998,2022-08-14 09:00:00,0.3865,0.3922,0.3864,0.3906,12183975.0


In [2]:
import pandas as pd
import numpy as np
# LOAD DATAS
#-=--=-=-=-=-=-=-=-=-==-=-=-=-=-=-=-=-=-=--=-=-=-=-=-=-
df = pd.read_csv("crypto.csv")
# remove special character from header
df.columns = df.columns.str.replace(' ', '')
lengthAll = len(df.index)

df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(by=['date'], ascending=True)
df = df.reset_index(drop=True)
display(df)
#-=--=-=-=-=-=-=-=-=-==-=-=-=-=-=-=-=-=-=--=-=-=-=-=-=-


feature = 'close'




#RSI CALCULATION
n = 14
def rma(x, n, y0):
    a = (n-1) / n
    ak = a**np.arange(len(x)-1, -1, -1)
    return np.r_[np.full(n, np.nan), y0, np.cumsum(ak * x) / ak / n + y0 * a**np.arange(1, len(x)+1)]

df['change'] = df[feature].diff()
df['gain'] = df.change.mask(df.change < 0, 0.0)
df['loss'] = -df.change.mask(df.change > 0, -0.0)
df['avg_gain'] = rma(df.gain[n+1:].to_numpy(), n, np.nansum(df.gain.to_numpy()[:n+1])/n)
df['avg_loss'] = rma(df.loss[n+1:].to_numpy(), n, np.nansum(df.loss.to_numpy()[:n+1])/n)
df['rs'] = df.avg_gain / df.avg_loss
df['rsi'] = 100 - (100 / (1 + df.rs))
# New formula for rsi end

# Save dataframe as CSV
df.to_csv('crypto_rsi.csv', index=False)



#SHIFT FORCASTING
import pandas as pd
import numpy as np
import random
from matplotlib import pyplot as plt 
from statsmodels.tsa.api import VAR

# if differenced only start
def invert_transformation(df_train, df_forecast, second_diff=False):
#     """Revert back the differencing to get the forecast to original scale."""
    df_fc = df_forecast.copy()
    columns = df_train.columns
    for col in columns:        
        # Roll back 2nd Diff
        if second_diff:
            df_fc[str(col)+'_1d'] = (df_train[col].iloc[-1]-df_train[col].iloc[-2]) + df_fc[str(col)+'_2d'].cumsum()
#         Roll back 1st Diff
        df_fc[str(col)+'_forecast'] = df_train[col].iloc[-1] + df_fc[str(col)+'_1d'].cumsum()
    return df_fc
# if differenced only end





# SET FEATURE TO FORECAST
features = ['open' , 'high', 'low', 'close', 'rsi', 'volume']


forecasted_array = []
true_array = []
time_array = []
invest_array = []
bank_array=[]
sum_differences_array=[]
forecast_all = []
time_forecast_all = []

# TAKE THREE MONTHs OF DATA TO PREDICT NEXT 1 HOUR
nr_points_to_consider = 200
bank = 100
how_many_hours_to_look = 600
lagorder = 20

for index, hour in df['date'].items():
    if index >= nr_points_to_consider and index < nr_points_to_consider+how_many_hours_to_look:
        print('percentage: ',(index-nr_points_to_consider)*100/(how_many_hours_to_look),"%")
        df_1 = df.iloc[(index-nr_points_to_consider):(index),:]
        df_1 = df_1.reset_index(drop=True)
        differenced_df = df_1[features].diff().dropna()
        model = VAR(differenced_df)
        model_fitted = model.fit(lagorder)
        # Get the lag order
        lag_order = model_fitted.k_ar
        forecast_input = differenced_df.values[-lag_order:]
        # Forecast
        step = lagorder
        fc = model_fitted.forecast(y=forecast_input, steps=step)
        df_forecast = pd.DataFrame(fc, index=df.index[nr_points_to_consider:nr_points_to_consider+step], columns=df_1[features].columns + '_1d')
        df_results  = invert_transformation(df_1[features], df_forecast , second_diff=False) 
        forecasted_array.append(df_results[feature+'_forecast'].values[0])
        if(index == nr_points_to_consider):
            forecast_all = df_results[feature+'_forecast'].tolist()
            time_forecast_all = df['date'].iloc[index:index+step]
        true_array.append(df[feature][index])
#         print('next point is=',df['date'][index])
        time_array.append(df['date'][index])
#         display(df_1)
#         display(df_results)
#         display(df.iloc[index:index+step])
#         print('index=',index)
        invest = False
        sum_diff = (df[feature][index] - df_results[feature+'_forecast'].values[0]) + (df_results[feature+'_forecast'].values[0] - df_results[feature+'_forecast'].values[1])           
        sum_differences_array.append(sum_diff)
#         print('=================================================================')
#         print('index=',index)
#         print('df_1')
#         display(df_1)
#         print('differenced_df')
#         display(differenced_df)
#         print('df_forecast')
#         display(differenced_df)
#         print('df_results')
#         display(df_results)
#         print('current time',df['date'][index])
        
        if(    
            (sum_diff>0 and df['rsi'][index-1] <70)
            and df['rsi'][index-3] < 30 
         ):
            invest = True
            bank = bank + bank*(df[feature][index]-df[feature][index-1])
        else:
            if invest:
                invest = False
        invest_array.append(invest)    
        bank_array.append(bank)     

        
# display(forecasted_array)
# display(time_array)
    
forecastedAndTure = pd.DataFrame(time_array,columns=['time'])
forecastedAndTure["forecast"] =  pd.DataFrame(forecasted_array,columns=['forecast'])
forecastedAndTure["true"] =  pd.DataFrame(true_array,columns=['true'])
forecastedAndTure["invest"] =  pd.DataFrame(invest_array,columns=['invest'])
forecastedAndTure["bank"] =  pd.DataFrame(bank_array,columns=['bank'])
forecastedAndTure["sum_diff"] =  pd.DataFrame(sum_differences_array,columns=['sum_diff'])
# print('df forecast=',forecastedAndTure["forecast"])
# print('df true=',forecastedAndTure["true"])
forecastedAndTure.to_csv('forecastedAndTure.csv', index=False)
display(forecastedAndTure)


#PLOTS
import plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go
#  shared_xaxes=True for together zoomin in same time
figure = make_subplots(rows=3, cols=1, shared_xaxes=True)
figure.update_layout(height=3*400, width=1100, title_text="All Subplots")

#plot current column of dataframe training
figure.append_trace(go.Scatter(
    x = forecastedAndTure.time,
    y = forecastedAndTure['forecast'].tolist(), 
    line=dict(width = 0.5),
#     fill='tozeroy',
    name = feature+' forecast',
    mode = 'markers'
), row=1, col=1)


#plot current column of dataframe
figure.append_trace(go.Scatter(
    x = df['date'].tolist(),
    y = df[feature].tolist(), 
    line=dict(width = 1.3),
#     fill='tozeroy',
    name = 'true all',
    mode = 'lines'
), row=1, col=1)


#plot current column of dataframe
figure.append_trace(go.Scatter(
    x = forecastedAndTure.time,
    y = forecastedAndTure['true'].tolist(), 
    line=dict(width = 0.7),
#     fill='tozeroy',
    name = feature+'  true',
    mode = 'lines'
), row=1, col=1)

#plot current column of dataframe
figure.append_trace(go.Scatter(
    x = df['date'].tolist(),
    y = df['rsi'].tolist(), 
    line=dict(width = 0.7),
#     fill='tozeroy',
    name = 'rsi',
    mode = 'lines'
), row=2, col=1)



#plot current column of dataframe training
figure.append_trace(go.Scatter(
    x = time_forecast_all.tolist(),
    y = forecast_all, 
    line=dict(width = 0.5),
#     fill='tozeroy',
    name = feature+' forecast step='+str(step),
    mode = 'markers'
), row=3, col=1)


#plot current column of dataframe
figure.append_trace(go.Scatter(
    x = df['date'].tolist(),
    y = df[feature].tolist(), 
    line=dict(width = 1.3),
#     fill='tozeroy',
    name = 'true all',
    mode = 'lines'
), row=3, col=1)


figure.add_hline(y=70, line_dash="dot",
              annotation_text="upper limit", 
              annotation_position="bottom right", row=2)

figure.add_hline(y=30, line_dash="dot",
              annotation_text="lower limit", 
              annotation_position="bottom right", row=2)

figure.update_xaxes(title_text='Time')
figure.update_yaxes(title_text=df[feature].name, row=1)
figure.update_xaxes(showticklabels=True)

# show the plot in this separate html
plotly.offline.plot(figure, filename='plots_crypto.html') 

,date,open,high,low,close,volume
0,2022-08-14 08:00:00,0.3866,0.3880,0.3849,0.3866,8629920.0
1,2022-08-14 09:00:00,0.3865,0.3922,0.3864,0.3906,12183975.0
2,2022-08-14 10:00:00,0.3906,0.3908,0.3870,0.3870,10021080.0
3,2022-08-14 11:00:00,0.3871,0.3886,0.3849,0.3850,8982403.0
4,2022-08-14 12:00:00,0.3850,0.3862,0.3834,0.3844,6493096.0
...,...,...,...,...,...,...
995,2022-09-24 19:00:00,0.5022,0.5029,0.4933,0.4980,26514808.0
996,2022-09-24 20:00:00,0.4979,0.5062,0.4959,0.5036,22271544.0
997,2022-09-24 21:00:00,0.5036,0.5085,0.4817,0.4946,45911444.0
998,2022-09-24 22:00:00,0.4945,0.4946,0.4824,0.4870,29456214.0


c:\users\buklla\appdata\local\programs\python\python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
c:\users\buklla\appdata\local\programs\python\python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
c:\users\buklla\appdata\local\programs\python\python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
c:\users\buklla\appdata\local\programs\python\python38\lib\site-p

percentage:  0.0 %
percentage:  0.16666666666666666 %
percentage:  0.3333333333333333 %
percentage:  0.5 %
percentage:  0.6666666666666666 %
percentage:  0.8333333333333334 %
percentage:  1.0 %


c:\users\buklla\appdata\local\programs\python\python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
c:\users\buklla\appdata\local\programs\python\python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
c:\users\buklla\appdata\local\programs\python\python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
c:\users\buklla\appdata\local\programs\python\python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasti

percentage:  1.1666666666666667 %
percentage:  1.3333333333333333 %
percentage:  1.5 %
percentage:  1.6666666666666667 %
percentage:  1.8333333333333333 %
percentage:  2.0 %
percentage:  2.1666666666666665 %


...

c:\users\buklla\appdata\local\programs\python\python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
c:\users\buklla\appdata\local\programs\python\python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
c:\users\buklla\appdata\local\programs\python\python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
c:\users\buklla\appdata\local\programs\python\python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasti

percentage:  99.16666666666667 %
percentage:  99.33333333333333 %
percentage:  99.5 %
percentage:  99.66666666666667 %
percentage:  99.83333333333333 %


c:\users\buklla\appdata\local\programs\python\python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'


,time,forecast,true,invest,bank,sum_diff
0,2022-08-22 16:00:00,0.336318,0.3367,False,100.000000,0.007135
1,2022-08-22 17:00:00,0.329655,0.3357,False,100.000000,0.000131
2,2022-08-22 18:00:00,0.339745,0.3342,False,100.000000,-0.000220
3,2022-08-22 19:00:00,0.330663,0.3350,False,100.000000,0.005602
4,2022-08-22 20:00:00,0.334828,0.3385,False,100.000000,0.010788
...,...,...,...,...,...,...
595,2022-09-16 11:00:00,0.331104,0.3302,False,99.657775,0.003319
596,2022-09-16 12:00:00,0.326923,0.3292,False,99.657775,-0.001296
597,2022-09-16 13:00:00,0.331486,0.3276,False,99.657775,-0.000914
598,2022-09-16 14:00:00,0.325792,0.3310,False,99.657775,0.006560


'plots_crypto.html'